# butcher4

The Markov Butcher Shop sells premium grade A+ beef. Customers arrive at the butcher shop and form a single queue. There is one butcher who serves customers from the queue on a first-come-first-served basis.

Based on historical data, the interarrival time between customers is uniformly distributed between 1.5 and 3.5 minutes. The time for a butcher to fill a single order is uniformly distributed between 2.5 and 6.5 minutes. The interarrival times and service times are assumed to be independent.

Simulate the shop for 2 hours, assuming there are $x$ identical butchers, where $x = 1, 2, 3, 4$. The butcher shop is interested in the following performance measures:

* the average time a customer spends in the queue,
* the time average number of customers in the queue,
* the time average number of busy butchers.

Determine a point estimate (i.e. observed sample mean) and an interval estimate (i.e. 95% confidence interval) for each performance measure using 50 replications.

Explain what is happening with your confidence intervals when $x = 4$.

#### Model

Below is the simulation model from the **butcher3** homework problem, except for the `model()` function, and with some additional functions imported.

In [1]:
##### Setup #####
# Import all simulation functions from SimPy
from SimPy.Simulation import *

# Import mean, std, and sqrt from NumPy
from numpy import mean, std, sqrt

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform

# Import t random variable from SciPy
from scipy.stats import t


##### Parameters #####
class P:
    # Customers arrive at the entrance with uniformly distributed
    # interarrival times between 1.5 and 3.5 minutes
    interarrivalTimeMin = 1.5
    interarrivalTimeMax = 3.5
    
    # Service times are uniformly distributed between 2.5 and 6.5 minutes
    serviceTimeMin = 2.5
    serviceTimeMax = 6.5
    
    # Number of servers
    nServers = None
    
    # Shop is open for 2 continuous hours
    simulationTimeMax =  2 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        arrivalTime = now()
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        M.delay.observe(now() - arrivalTime)
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = uniform(low = P.interarrivalTimeMin, high = P.interarrivalTimeMax)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

##### Resources #####
class R:
    # Server
    server = None
    

##### Monitors #####
class M:
    # Queue waiting time
    delay = None

Here is the `model()` function, modified to:

* take a seed value as input,
* output a **list** of the observed performance measures.

Having `model()` output a list of observed performance measures, instead of just one, lets us simultaneously estimate the three performance measures.

In [2]:
##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator (more on this later in the semester)
    seed(inputSeed)

    # Create the server resource
    R.server = Resource(capacity=P.nServers, monitored=True)
    
    # Create the delay monitor
    M.delay = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until=P.simulationTimeMax)
    
    # Compute average delay
    avgDelay = M.delay.mean()
    
    # Compute time average number of customers in queue
    tAvgCustQ = R.server.waitMon.timeAverage()
    
    # Compute time average number of busy butchers
    tAvgBusyS = R.server.actMon.timeAverage()
    
    # Return a list of performance measures
    return [avgDelay, tAvgCustQ, tAvgBusyS]

#### 1 butcher

Let's start with the case of 1 butcher.

In [3]:
# Set number of butchers
P.nServers = 1

# Replicate simulation 50 times
# Create a list of observations for each performance measure
n = 50
avgDelayObs = []
tAvgCustQObs = []
tAvgBusySObs = []
for i in range(n):
    [avgDelay, tAvgCustQ, tAvgBusyS] = model(i*123)
    avgDelayObs.append(avgDelay)
    tAvgCustQObs.append(tAvgCustQ)
    tAvgBusySObs.append(tAvgBusyS)

    
#### Average delay ####
# Observed sample mean
avgDelaySM = mean(avgDelayObs)

# Observed sample standard deviation
avgDelaySSD = std(avgDelayObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelayCIL = avgDelaySM - t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)
avgDelayCIR = avgDelaySM + t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)

print("Average delay:")
print("  Sample mean: {0}".format(avgDelaySM))
print("  Sample standard deviation: {0}".format(avgDelaySSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayCIL, avgDelayCIR))


#### Time average number of customers in queue ####
# Observed sample mean
tAvgCustQSM = mean(tAvgCustQObs)

# Observed sample standard deviation
tAvgCustQSSD = std(tAvgCustQObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgCustQCIL = tAvgCustQSM - t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)
tAvgCustQCIR = tAvgCustQSM + t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)

print("Time average number of customers in queue:")
print("  Sample mean: {0}".format(tAvgCustQSM))
print("  Sample standard deviation: {0}".format(tAvgCustQSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgCustQCIL, tAvgCustQCIR))


#### Time average number of busy servers ####
# Observed sample mean
tAvgBusySSM = mean(tAvgBusySObs)

# Observed sample standard deviation
tAvgBusySSSD = std(tAvgBusySObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgBusySCIL = tAvgBusySSM - t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)
tAvgBusySCIR = tAvgBusySSM + t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)

print("Time average number of busy servers:")
print("  Sample mean: {0}".format(tAvgBusySSM))
print("  Sample standard deviation: {0}".format(tAvgBusySSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgBusySCIL, tAvgBusySCIR))

Average delay:
  Sample mean: 25.33078281767246
  Sample standard deviation: 2.38958385107669
  95.0% confidence interval: [24.651670602145256, 26.009895033199665]
Time average number of customers in queue:
  Sample mean: 10.135182453478603
  Sample standard deviation: 1.0568862312040057
  95.0% confidence interval: [9.834818710348287, 10.435546196608918]
Time average number of busy servers:
  Sample mean: 0.9795047836641766
  Sample standard deviation: 0.00450698501933938
  95.0% confidence interval: [0.9782239126955967, 0.9807856546327565]


#### 2 butchers

In [4]:
# Set number of butchers
P.nServers = 2

# Replicate simulation 50 times
# Create a list of observations for each performance measure
n = 50
avgDelayObs = []
tAvgCustQObs = []
tAvgBusySObs = []
for i in range(n):
    [avgDelay, tAvgCustQ, tAvgBusyS] = model(i*123)
    avgDelayObs.append(avgDelay)
    tAvgCustQObs.append(tAvgCustQ)
    tAvgBusySObs.append(tAvgBusyS)

    
#### Average delay ####
# Observed sample mean
avgDelaySM = mean(avgDelayObs)

# Observed sample standard deviation
avgDelaySSD = std(avgDelayObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelayCIL = avgDelaySM - t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)
avgDelayCIR = avgDelaySM + t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)

print("Average delay:")
print("  Sample mean: {0}".format(avgDelaySM))
print("  Sample standard deviation: {0}".format(avgDelaySSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayCIL, avgDelayCIR))


#### Time average number of customers in queue ####
# Observed sample mean
tAvgCustQSM = mean(tAvgCustQObs)

# Observed sample standard deviation
tAvgCustQSSD = std(tAvgCustQObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgCustQCIL = tAvgCustQSM - t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)
tAvgCustQCIR = tAvgCustQSM + t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)

print("Time average number of customers in queue:")
print("  Sample mean: {0}".format(tAvgCustQSM))
print("  Sample standard deviation: {0}".format(tAvgCustQSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgCustQCIL, tAvgCustQCIR))


#### Time average number of busy servers ####
# Observed sample mean
tAvgBusySSM = mean(tAvgBusySObs)

# Observed sample standard deviation
tAvgBusySSSD = std(tAvgBusySObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgBusySCIL = tAvgBusySSM - t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)
tAvgBusySCIR = tAvgBusySSM + t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)

print("Time average number of busy servers:")
print("  Sample mean: {0}".format(tAvgBusySSM))
print("  Sample standard deviation: {0}".format(tAvgBusySSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgBusySCIL, tAvgBusySCIR))

Average delay:
  Sample mean: 0.760729761516865
  Sample standard deviation: 0.6160774863591819
  95.0% confidence interval: [0.5856424773818887, 0.9358170456518413]
Time average number of customers in queue:
  Sample mean: 0.3087607694548542
  Sample standard deviation: 0.2685584836480793
  95.0% confidence interval: [0.23243729298563676, 0.38508424592407164]
Time average number of busy servers:
  Sample mean: 1.7322644980552713
  Sample standard deviation: 0.08287428805140581
  95.0% confidence interval: [1.7087118860205561, 1.7558171100899864]


#### 3 butchers

In [5]:
# Set number of butchers
P.nServers = 3

# Replicate simulation 50 times
# Create a list of observations for each performance measure
n = 50
avgDelayObs = []
tAvgCustQObs = []
tAvgBusySObs = []
for i in range(n):
    [avgDelay, tAvgCustQ, tAvgBusyS] = model(i*123)
    avgDelayObs.append(avgDelay)
    tAvgCustQObs.append(tAvgCustQ)
    tAvgBusySObs.append(tAvgBusyS)

    
#### Average delay ####
# Observed sample mean
avgDelaySM = mean(avgDelayObs)

# Observed sample standard deviation
avgDelaySSD = std(avgDelayObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelayCIL = avgDelaySM - t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)
avgDelayCIR = avgDelaySM + t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)

print("Average delay:")
print("  Sample mean: {0}".format(avgDelaySM))
print("  Sample standard deviation: {0}".format(avgDelaySSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayCIL, avgDelayCIR))


#### Time average number of customers in queue ####
# Observed sample mean
tAvgCustQSM = mean(tAvgCustQObs)

# Observed sample standard deviation
tAvgCustQSSD = std(tAvgCustQObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgCustQCIL = tAvgCustQSM - t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)
tAvgCustQCIR = tAvgCustQSM + t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)

print("Time average number of customers in queue:")
print("  Sample mean: {0}".format(tAvgCustQSM))
print("  Sample standard deviation: {0}".format(tAvgCustQSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgCustQCIL, tAvgCustQCIR))


#### Time average number of busy servers ####
# Observed sample mean
tAvgBusySSM = mean(tAvgBusySObs)

# Observed sample standard deviation
tAvgBusySSSD = std(tAvgBusySObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgBusySCIL = tAvgBusySSM - t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)
tAvgBusySCIR = tAvgBusySSM + t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)

print("Time average number of busy servers:")
print("  Sample mean: {0}".format(tAvgBusySSM))
print("  Sample standard deviation: {0}".format(tAvgBusySSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgBusySCIL, tAvgBusySCIR))

Average delay:
  Sample mean: 0.00550230578084469
  Sample standard deviation: 0.008378058603183998
  95.0% confidence interval: [0.003121287873788855, 0.007883323687900526]
Time average number of customers in queue:
  Sample mean: 0.002249976648890206
  Sample standard deviation: 0.0034699541347685678
  95.0% confidence interval: [0.001263826596370104, 0.0032361267014103085]
Time average number of busy servers:
  Sample mean: 1.748797424247714
  Sample standard deviation: 0.09318944764176845
  95.0% confidence interval: [1.7223132762973323, 1.7752815721980957]


#### 4 butchers

In [6]:
# Set number of butchers
P.nServers = 4

# Replicate simulation 50 times
# Create a list of observations for each performance measure
n = 50
avgDelayObs = []
tAvgCustQObs = []
tAvgBusySObs = []
for i in range(n):
    [avgDelay, tAvgCustQ, tAvgBusyS] = model(i*123)
    avgDelayObs.append(avgDelay)
    tAvgCustQObs.append(tAvgCustQ)
    tAvgBusySObs.append(tAvgBusyS)

    
#### Average delay ####
# Observed sample mean
avgDelaySM = mean(avgDelayObs)

# Observed sample standard deviation
avgDelaySSD = std(avgDelayObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelayCIL = avgDelaySM - t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)
avgDelayCIR = avgDelaySM + t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)

print("Average delay:")
print("  Sample mean: {0}".format(avgDelaySM))
print("  Sample standard deviation: {0}".format(avgDelaySSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayCIL, avgDelayCIR))


#### Time average number of customers in queue ####
# Observed sample mean
tAvgCustQSM = mean(tAvgCustQObs)

# Observed sample standard deviation
tAvgCustQSSD = std(tAvgCustQObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgCustQCIL = tAvgCustQSM - t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)
tAvgCustQCIR = tAvgCustQSM + t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)

print("Time average number of customers in queue:")
print("  Sample mean: {0}".format(tAvgCustQSM))
print("  Sample standard deviation: {0}".format(tAvgCustQSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgCustQCIL, tAvgCustQCIR))


#### Time average number of busy servers ####
# Observed sample mean
tAvgBusySSM = mean(tAvgBusySObs)

# Observed sample standard deviation
tAvgBusySSSD = std(tAvgBusySObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgBusySCIL = tAvgBusySSM - t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)
tAvgBusySCIR = tAvgBusySSM + t.ppf(1 - alpha/2, n - 1) * tAvgBusySSSD / sqrt(n)

print("Time average number of busy servers:")
print("  Sample mean: {0}".format(tAvgBusySSM))
print("  Sample standard deviation: {0}".format(tAvgBusySSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgBusySCIL, tAvgBusySCIR))

Average delay:
  Sample mean: 0.0
  Sample standard deviation: 0.0
  95.0% confidence interval: [0.0, 0.0]
Time average number of customers in queue:
  Sample mean: 0.0
  Sample standard deviation: 0.0
  95.0% confidence interval: [0.0, 0.0]
Time average number of busy servers:
  Sample mean: 1.7489067938375376
  Sample standard deviation: 0.09314575991116401
  95.0% confidence interval: [1.722435061802801, 1.775378525872274]


When there are 4 butchers, no customers experience delay, and as a result, the average delay and time average number of customers in the queue from *every* simulation replication is simply 0. Therefore, the confidence intervals for the estimates of mean average delay and mean time average number of customers should have length 0.